In [1]:
from sentence_transformers import SentenceTransformer
import os
from deeplake.core.vectorstore import VectorStore

In [2]:
project_name = 'OpenVoice_results'

In [3]:
#Utils to extract pylint codes from pylint documentation

with open("pylint_doc.txt", "r") as file:
    lines = file.readlines()
    
pylint_codes=open("pylint_codes.txt","w")
codes=[]

for line in lines:
    if line.startswith(":"):
        #find the first word beween ( and )
        code=line[line.find("(")+1:line.find(")")]
        pylint_codes.write(code+"\n")
        codes.append(code)
        
pylint_codes.close()
# print(lines)


In [4]:
flag=True
if not os.path.exists(f"input_lines_dbms_{project_name}.txt"):
    metadata=[]
    lines_for_dbms=[]
    final_report=open(f"input_lines_dbms_{project_name}.txt","w")


    # Loop through each folder from v0 to v99
    for i in range(100):
        folder_name = f'v{i}'
        file_path = os.path.join(project_name, folder_name, 'pylint.txt')
        print(file_path)
        
        # Check if the data.txt file exists
        if os.path.isfile(file_path):
            # Open and read the file
            with open(file_path, 'r') as file:
                report_lines = file.readlines()
                
            for line in report_lines:
                try:
                    code= line.split(" ")[1][:-1]
                    file_name=line.split(" ")[0][:line.split(" ")[0].find(":")]
                    version=folder_name
                    
                    #add version to line
                    
                    new_line=folder_name+": "+line
                    
                    if code in codes:
                        lines_for_dbms.append(new_line)
                        final_report.write(new_line)
                        temp={"code":code,"file_name":file_name, "version":version}
                        metadata.append(temp)
                        
                except:
                    continue
            
                
        else:
            print(f"File not found: {file_path}")


    final_report.close()
    print(f"total number of lines: {len(lines_for_dbms)}")
else: 
    print("File already exists")
    print("Database is created already")
    flag=False

File already exists
Database is created already


In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L12-v2")
#for gpt-soVITs change model to all-mpnet-base-v2 else for others all-MiniLM-L12-v2


In [6]:
def embed(sentence):
    return model.encode(sentence)

In [7]:
### create new database
from deeplake.core.vectorstore import VectorStore

In [8]:
vector_store_path="vectordb"+project_name

vector_db=VectorStore(path=vector_store_path)

Deep Lake Dataset in vectordbOpenVoice_results already exists, loading from the storage


In [9]:
vector_db.summary()

Dataset(path='vectordbOpenVoice_results', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
 embedding  embedding  (57202, 384)  float32   None   
    id        text      (57202, 1)     str     None   
 metadata     json      (57202, 1)     str     None   
   text       text      (57202, 1)     str     None   


In [10]:
if (vector_db.__len__()==0 and flag==True):
    embeddings=model.encode(sentences=lines_for_dbms)
    print(len(embeddings))
    vector_db.add(text=lines_for_dbms, embedding=embeddings, metadata=metadata)
elif flag==False and vector_db.__len__()==0:
    print(f"Database should already exist, delete input_lines_dbms_{project_name}.txt and rerun the code")
    
else: 
    print("Database already exists!")


Database already exists!


In [11]:
def query(query, number_of_suggestions=10, version_number=None, file_name=None, error_code=None):
    filter={}
    if version_number:
        filter['version']=version_number
    if file_name:
        filter['file_name']=file_name
    if error_code:
        filter['code']=error_code
        
    if filter=={}:
        search_result=vector_db.search(embedding_data=query,exec_option="python",embedding_function=embed, k=number_of_suggestions)
        return search_result
    
    search_result=vector_db.search(embedding_data=query,exec_option="python",embedding_function=embed, k=number_of_suggestions, filter={"metadata":filter})
    
    return search_result

In [12]:
def print_results(dict):
    for i in dict["text"]: 
        print(i)
        


In [13]:
q1=query("names", 10)
print_results(q1)


v27: OpenVoice_results/v27/text/symbols.py:78:0: C0103: Constant name "num_ja_tones" doesn't conform to UPPER_CASE naming style (invalid-name)

v39: OpenVoice_results/v39/text/symbols.py:78:0: C0103: Constant name "num_ja_tones" doesn't conform to UPPER_CASE naming style (invalid-name)

v42: OpenVoice_results/v42/text/symbols.py:78:0: C0103: Constant name "num_ja_tones" doesn't conform to UPPER_CASE naming style (invalid-name)

v20: OpenVoice_results/v20/text/symbols.py:78:0: C0103: Constant name "num_ja_tones" doesn't conform to UPPER_CASE naming style (invalid-name)

v83: OpenVoice_results/v83/openvoice/text/symbols.py:78:0: C0103: Constant name "num_ja_tones" doesn't conform to UPPER_CASE naming style (invalid-name)

v41: OpenVoice_results/v41/text/symbols.py:78:0: C0103: Constant name "num_ja_tones" doesn't conform to UPPER_CASE naming style (invalid-name)

v58: OpenVoice_results/v58/text/symbols.py:78:0: C0103: Constant name "num_ja_tones" doesn't conform to UPPER_CASE naming styl

In [14]:
q1

{'score': [0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812,
  0.2407953143119812],
 'id': ['bc0813e6-0627-11ef-80c3-ca200fd864c9',
  'bc09ffa8-0627-11ef-80c3-ca200fd864c9',
  'bc0a7a6e-0627-11ef-80c3-ca200fd864c9',
  'bc06eef8-0627-11ef-80c3-ca200fd864c9',
  'bc1197f4-0627-11ef-80c3-ca200fd864c9',
  'bc0a50fc-0627-11ef-80c3-ca200fd864c9',
  'bc0d471c-0627-11ef-80c3-ca200fd864c9',
  'bc0ea7f6-0627-11ef-80c3-ca200fd864c9',
  'bc0c10b8-0627-11ef-80c3-ca200fd864c9',
  'bc0f2e6a-0627-11ef-80c3-ca200fd864c9'],
 'metadata': [{'code': 'C0103',
   'file_name': 'OpenVoice_results/v27/text/symbols.py',
   'version': 'v27'},
  {'code': 'C0103',
   'file_name': 'OpenVoice_results/v39/text/symbols.py',
   'version': 'v39'},
  {'code': 'C0103',
   'file_name': 'OpenVoice_results/v42/text/symbols.py',
   'version': 'v42'},
  {'code': 'C0103',
   'file_name'

In [20]:
q2=query("module import", version_number="v29", number_of_suggestions=30)
print_results(q2)



100%|██████████| 57202/57202 [00:11<00:00, 4809.28it/s]


v29: OpenVoice_results/v29/models.py:7:0: E0401: Unable to import 'modules' (import-error)

v29: OpenVoice_results/v29/text/english.py:20:0: C0413: Import "import inflect" should be placed at the top of the module (wrong-import-position)

v29: OpenVoice_results/v29/text/english.py:19:0: C0413: Import "import re" should be placed at the top of the module (wrong-import-position)

v29: OpenVoice_results/v29/text/english.py:21:0: C0413: Import "from unidecode import unidecode" should be placed at the top of the module (wrong-import-position)

v29: OpenVoice_results/v29/text/english.py:22:0: C0413: Import "import eng_to_ipa as ipa" should be placed at the top of the module (wrong-import-position)

v29: OpenVoice_results/v29/text/__init__.py:46:0: C0413: Import "from text.symbols import language_tone_start_map" should be placed at the top of the module (wrong-import-position)

v29: OpenVoice_results/v29/api.py:3:0: C0411: standard import "re" should be placed before third party imports "torc

In [21]:
q3=query("are too many if else causing issues")
print_results(q3)

v36: OpenVoice_results/v36/api.py:95:8: R1705: Unnecessary "else" after "return", remove the "else" and de-indent the code inside it (no-else-return)

v29: OpenVoice_results/v29/api.py:95:8: R1705: Unnecessary "else" after "return", remove the "else" and de-indent the code inside it (no-else-return)

v94: OpenVoice_results/v94/openvoice/modules.py:513:8: R1705: Unnecessary "else" after "return", remove the "else" and de-indent the code inside it (no-else-return)

v74: OpenVoice_results/v74/modules.py:365:8: R1705: Unnecessary "else" after "return", remove the "else" and de-indent the code inside it (no-else-return)

v33: OpenVoice_results/v33/modules.py:448:8: R1705: Unnecessary "else" after "return", remove the "else" and de-indent the code inside it (no-else-return)

v74: OpenVoice_results/v74/modules.py:377:8: R1705: Unnecessary "else" after "return", remove the "else" and de-indent the code inside it (no-else-return)

v94: OpenVoice_results/v94/openvoice/modules.py:573:8: R1705: Un

In [22]:
q4=query("length related problems")
print_results(q4)

v75: OpenVoice_results/v75/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v53: OpenVoice_results/v53/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v92: OpenVoice_results/v92/openvoice/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v56: OpenVoice_results/v56/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v77: OpenVoice_results/v77/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v55: OpenVoice_results/v55/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v84: OpenVoice_results/v84/openvoice/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v85: OpenVoice_results/v85/openvoice/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v81: OpenVoice_results/v81/openvoice/openvoice_app.py:108:0: C0301: Line too long (141/100) (line-too-long)

v66: OpenVoice_results/v66/openvoice_app.py:108:0: C0301: Line too lo

In [25]:
q4=query("import related problems", file_name="OpenVoice_results/v66/openvoice_app.py", version_number="v66")
print_results(q4)

100%|██████████| 57202/57202 [00:11<00:00, 4999.09it/s]

v66: OpenVoice_results/v66/openvoice_app.py:7:0: E0401: Unable to import 'se_extractor' (import-error)

v66: OpenVoice_results/v66/openvoice_app.py:6:0: E0401: Unable to import 'langid' (import-error)

v66: OpenVoice_results/v66/openvoice_app.py:5:0: C0411: standard import "zipfile.ZipFile" should be placed before third party imports "torch", "gradio" (wrong-import-order)

v66: OpenVoice_results/v66/openvoice_app.py:8:0: E0401: Unable to import 'api' (import-error)

v66: OpenVoice_results/v66/openvoice_app.py:3:0: C0411: standard import "argparse" should be placed before third party import "torch" (wrong-import-order)

v66: OpenVoice_results/v66/openvoice_app.py:5:0: W0611: Unused ZipFile imported from zipfile (unused-import)

v66: OpenVoice_results/v66/openvoice_app.py:4:0: E0401: Unable to import 'gradio' (import-error)

v66: OpenVoice_results/v66/openvoice_app.py:1:0: C0114: Missing module docstring (missing-module-docstring)

v66: OpenVoice_results/v66/openvoice_app.py:37:0: R0914:

In [ ]:
q4